In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

x = pd.read_excel('')
x['Churn'] = x['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
x['MaritalStatus'] = pd.get_dummies(x['MaritalStatus'])['Yes']

x.shape
# Categorizing two features: 16.

# 1 feature

plt.hist(x['MonthlyMinutes'], bins=int(np.round(np.sqrt(x.shape[0]))))
plt.title("Histogram of first feature")
plt.show()
# boundaries for four categories of first feature : [0, 500, 900,4000]
plt.hist(x['OutboundCalls'], bins=int(np.round(np.sqrt(x.shape[0]))))
plt.title("Histogram of second feature")
plt.show()
# boundaries for four categories of second feature : [0, 15, 30, 55,280]
parts_names = ['Professional', "Retired", "Self", "Student"]
# take that values in nominal feature Occupation,
# as that values are approximately 80% jf all data

# create 2 nominal features from MonthlyMinutes
# and OutboundCalls

x['new_nominal_f1'] = 0
for i in range(x.shape[0]):
    x['new_nominal_f2'] = 0

    if 0 <= x.iloc[i, 1] <= 500:
        x.iloc[i, -1] = 1
    elif 500 < x.iloc[i, 1] <= 900:
        x.iloc[i, -1] = 2
    else:
        x.iloc[i, -1] = 3

x['new_nominal_f2'] = 0
for i in range(x.shape[0]):
    if 0 <= x.iloc[i, 3] <= 15:
        x.iloc[i, -1] = 1
    elif 15 < x.iloc[i, 3] <= 30:
        x.iloc[i, -1] = 2
    elif 30 < x.iloc[i, 3] <= 55:
        x.iloc[i, -1] = 3
    else:
        x.iloc[i, -1] = 4

x_ = np.array(x.copy())
x_ = x_[np.isin(x_[:, -3], parts_names)]


def contingency_table(subset_1, subset_2):
    """
     An slower yet more intuitive way to create contingency table.
     returns contingency tables of two subsets,
     so that the last column and last row represent the
     marginal frequencies of Tk and Gl.
      """
    cat_1, cat_1_idx = np.unique(subset_1, return_inverse=True)
    cat_2, cat_2_idx = np.unique(subset_2, return_inverse=True)
    n_cat_1 = cat_1.shape[0]
    n_cat_2 = cat_2.shape[0]
    contingency = np.zeros([n_cat_1 + 1, n_cat_2 + 1], dtype=np.int)

    for row in range(n_cat_1):
        for col in range(n_cat_2):
            n_rc = len(set(np.where(cat_1_idx == row)[0].tolist()).intersection(
                set(np.where(cat_2_idx == col)[0].tolist())))
            contingency[row, col] = n_rc

        contingency[row, -1] = np.sum(contingency[row, :])

    contingency[-1, :] = np.sum(contingency, axis=0)
    relative_frequencies = np.divide(contingency, contingency[-1, -1])
    return contingency, relative_frequencies


# create the contingency tables
contingency_T_G, rel_freq_T_G = contingency_table(x_[:, -2], x_[:, -1])
print("contingency P(T|G):", "\n", )
print(contingency_T_G)
print(" ")
print("prob P(T|G):", "\n", )
print(rel_freq_T_G)

contingency_T_H, rel_freq_T_H = contingency_table(x_[:, -2], x_[:, -3])  # TgivenH
print("contingency P(T|H):", "\n", )
print(contingency_T_H)
print(" ")
print("prob P(T|H):", "\n", )

marg_freq_T = contingency_T_G[:-1, -1:]
marg_freq_G = contingency_T_G[-1, :-1]
marg_freq_H = contingency_T_H[-1, :-1]

conditional_T_G = np.divide(contingency_T_G[:-1, :-1], marg_freq_G)
print("Conditional T given G:", "\n \n", conditional_T_G)

conditional_T_H = np.divide(contingency_T_H[:-1, :-1], marg_freq_H)
print("Conditional T given H:", "\n \n", conditional_T_H)

quetelet_T_G = np.subtract(np.divide(conditional_T_G, marg_freq_T / x_.shape[0]), 1)
quetelet_T_G

quetelet_T_H = np.subtract(np.divide(conditional_T_H, marg_freq_T / x_.shape[0]), 1)
quetelet_T_H


In [ ]:

def compute_chi2(rel_freq, indep_prob):
    "As per slides page 50"
    d = np.subtract(rel_freq[:-1, :-1], indep_prob)
    dd = np.power(d, 2)
    chi = np.divide(dd, indep_T_n_G)
    chi2 = np.sum(chi)
    return chi2


# Observed relative contingency table and the table of Quetelet indices
ave_quet_idx_T_G = np.sum(rel_freq_T_G[:-1, :-1].reshape(1, -1) * quetelet_T_G.reshape(1, -1))
print("average Quetelet Index T given G:", ave_quet_idx_T_G)

# Independent probabilities:
indep_T_n_G = marg_freq_T / x_.shape[0] @ marg_freq_G.reshape(1, -1) / x_.shape[0]
indep_T_n_H = marg_freq_T / x_.shape[0] @ marg_freq_H.reshape(1, -1) / x_.shape[0]
chi2_T_G = compute_chi2(rel_freq_T_G, indep_T_n_G)
print("Chi2:", ave_quet_idx_T_G)

print('at N > ' + str(
    12.592 // 0.55425 + 1) + ' the hypothesis of statistical independence should be rejected at 95% confidence level.')
print('at N > ' + str(
    22.458 // 0.55425 + 1) + ' the hypothesis of statistical independence should be rejected at 99% confidence level.')
print('we have N = 432 => not ind')
ave_quet_idx_T_H = np.sum(rel_freq_T_H[:-1, :-1].reshape(1, -1) * quetelet_T_H.reshape(1, -1))
print("average Quetelet Index T given H:", ave_quet_idx_T_H)

chi2_T_H = compute_chi2(rel_freq_T_H, indep_T_n_H)
print("Chi2:", ave_quet_idx_T_H)

print('at N > ' + str(
    12.592 // 0.0164 + 1) + ' the hypothesis of statistical independence should be rejected at 95% confidence level.')
print('at N > ' + str(
    22.458 // 0.0164 + 1) + ' the hypothesis of statistical independence should be rejected at 99% confidence level.')
print('we have N = 432 => ind')

heat_map_quet_T_G = (rel_freq_T_G[:-1, :-1].reshape(1, -1) * quetelet_T_G.reshape(1, -1)).reshape(3, 4)
heat_map_quet_T_G

plt.imshow(heat_map_quet_T_G, cmap='hot',
           interpolation='nearest')
plt.colorbar()
plt.show()

heat_map_quet_T_H = (rel_freq_T_H[:-1, :-1].reshape(1, -1) * quetelet_T_H.reshape(1, -1)).reshape(3, 4)
heat_map_quet_T_H

plt.imshow(heat_map_quet_T_H, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()